# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp-adapters`.

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

OPENAI_API_KEY=****here
GROQ_API_KEY=****fKCN


In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [ ]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant to a banker. You help with timezone-aware time calculations and conversions.",
)

Ask about the time in another location

In [4]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "I want to do a wire transfer from India to USA at 9 AM IST tomorrow. What time will it be in New York?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

I want to do a wire transfer from India to USA at 9 AM IST tomorrow. What time will it be in New York?
================================== Ai Message ==================================
Tool Calls:
  convert_timezone (8asytcf7w)
 Call ID: 8asytcf7w
  Args:
    input_timezone: Asia/Kolkata
    output_timezone: America/New_York
    time: 2024-03-16T09:00:00Z
================================= Tool Message =================================
Name: convert_timezone

2024-03-16T05:00:00Z
================================== Ai Message ==================================

The time in New York would be 5 AM EST (Eastern Standard Time) tomorrow.
